**Get the data for the model training**

In this part of the code we will import the RNA-seq and CITE-seq data and start exploring potential models. First we would get the data in and convert it into the standard data format.

In [3]:
# install right libraries

!pip install scanpy --quiet


In [1]:

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc



### Import the data and convert

In [7]:

# protein expression data
pd_prot_exp = pd.read_csv('/home/ec2-user/modal_form/CrossModalFormer_CITE_Seq/Data/documentation/Protein_expression.csv')


In [ ]:

# RNA expression data
adata_1 = pd.read_csv('/home/ec2-user/modal_form/CrossModalFormer_CITE_Seq/Data/expression/RNA_expression_subset1a.csv')

### Convert RNA-seq data